# v3 - TAP modular (mat, geom, ... in a tools file)
- Date: 29/03/2023
- Based on: Zsolts serpent model (in TAPreactor folder), also based on v2
- Changelog:
    - 29 march 2023: initial version


Questions:
1. ...


In [1]:
# Fuel properties
FUEL_TEMP = 900 # K
# FUEL_ENRICHMENT = 4.0 # wo% # NOT NEEDED IN THIS MODEL
# FUEL_R = 5 # cm # NOT NEEDED IN THIS MODEL

# Moderator properties
MODERATOR_TEMP = 600 # K
MODERATOR_R = 1.15 # cm

# Cladding properties
CLADDING_TEMP = 900 # K
CLADDING_R = 1.25 # cm

# Lattice properties
LATTICE_ELEMENTS_INNER = 7
LATTICE_PITCH_INNER = 3 # cm

LATTICE_ELEMENTS_OUTER = 20 # Amount of elements per side
LATTICE_PITCH_OUTER = 15 # cm

# Containment properties
CONTAINMENT_R = 150 # cm
# CONTAINMENT_THICKNESS = 0 # cm # NOT NEEDED IN THIS MODEL

# Plenum properties
PLENUM_HEIGHT = 25 # cm

# Core properties
CORE_HEIGHT = 350 # cm

# Run properties
INACTIVE_BATCHES = 50
PARTICLE_COUNT = 30000
ACTIVE_BATCH_COUNT = 300
# FAST_REACTOR = True # True if epithermal, False if thermal
# EIGHTH = False
# QUARTER = False

DO_PLOT = True

In [4]:
import os
import sys
TOOLS_PATH = f"{os.getenv('MASTER_PROJECT_ROOT_FOLDER')}/logistics"
sys.path.insert(0, TOOLS_PATH)
from tools import model_tools
import openmc
import numpy as np
import matplotlib.pyplot as plt
import importlib
importlib.reload(model_tools)

for REACTOR_MODEL in [model_tools.THERMAL_REACTOR, model_tools.EPITHERMAL_REACTOR, model_tools.SEMIEPITHERMAL_REACTOR]:
    print(REACTOR_MODEL)

    # Materials
    material_fuel, material_moderator, material_cladding = model_tools.get_materials(FUEL_TEMP, CLADDING_TEMP, MODERATOR_TEMP)

    # Pin universe
    universe_pin, universe_outer, cell_fuel, cell_moderator, cell_cladding = model_tools.get_pin_universe(CORE_HEIGHT, MODERATOR_R, CLADDING_R, material_fuel, material_moderator, material_cladding)

    # Inner lattice
    if REACTOR_MODEL == model_tools.THERMAL_REACTOR or REACTOR_MODEL == model_tools.EPITHERMAL_REACTOR:
        universe_lattice_inner = model_tools.get_inner_lattice(LATTICE_PITCH_INNER, LATTICE_ELEMENTS_INNER, REACTOR_MODEL, universe_pin, universe_outer)
        universe_lattice_inner_2 = None
    elif REACTOR_MODEL == model_tools.SEMIEPITHERMAL_REACTOR:
        universe_lattice_inner_thermal = model_tools.get_inner_lattice(LATTICE_PITCH_INNER, LATTICE_ELEMENTS_INNER, model_tools.THERMAL_REACTOR, universe_pin, universe_outer)
        universe_lattice_inner_epithermal = model_tools.get_inner_lattice(LATTICE_PITCH_INNER, LATTICE_ELEMENTS_INNER, model_tools.EPITHERMAL_REACTOR, universe_pin, universe_outer)
        universe_lattice_inner = universe_lattice_inner_thermal
        universe_lattice_inner_2 = universe_lattice_inner_epithermal
    else:
        raise Exception("Invalid reactor model")

    # Outer lattice
    geometry, cell_main, cell_plenum = model_tools.get_outer_lattice(LATTICE_PITCH_OUTER, LATTICE_ELEMENTS_OUTER, CORE_HEIGHT, 
                      PLENUM_HEIGHT, CONTAINMENT_R, REACTOR_MODEL, universe_outer, 
                      universe_lattice_inner, material_fuel,
                      universe_lattice_inner_2=universe_lattice_inner_2)

    if DO_PLOT:
        model_tools.get_plots(REACTOR_MODEL, geometry, material_fuel, material_moderator, material_cladding)

THERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)


EPITHERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)


SEMIEPITHERMAL


/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/fne23_stjarnholm/miniconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
